In [0]:
%sql
create table if not exists sandbox.z_jeongjoo_lee.heds_1266_iot_user_cnt_dev as (

WITH platform_id_mapping AS (
  -- Get platform_id from normal_log tables
  SELECT DISTINCT 
    X_Device_Platform AS platform_id,
    X_Device_Platform AS platform_code
  FROM (
    select distinct platform_code as X_Device_Platform
    from kic_data_fact.common_tv.tv_usage_monthly
  )
  WHERE X_Device_Platform IS NOT NULL
),

platform_mapping AS (
  -- Get platform_code to platform_version mapping
  SELECT DISTINCT 
    platform_code,
    platform_version
  FROM kic_data_mart.tv_usage.iot_usage
  WHERE date_ym IN ('2025-11', '2025-12', '2026-01')
    AND platform_version IN ('webOS22', 'webOS23', 'webOS24', 'webOS25')
  -- WHERE date_ym IN ('2025-12', '2026-01')
  --   AND platform_version IN ( 'webOS25')
),

monthly_usage AS (
  -- Get monthly activated TVs from tv_usage_monthly
  SELECT 
    m.date_ym,
    m.country_code,
    m.platform_code,
    p.platform_version,
    SUM(m.one_day_plus_device) AS total_activated_tvs
  FROM kic_data_fact.common_tv.tv_usage_monthly m
  INNER JOIN platform_mapping p ON m.platform_code = p.platform_code
  WHERE m.date_ym IN ('2025-11', '2025-12', '2026-01')
  -- WHERE m.date_ym IN ('2025-12', '2026-01')
  GROUP BY m.date_ym, m.country_code, m.platform_code, p.platform_version
),

iot_metrics AS (
  -- Get IoT device usage metrics
  SELECT 
    date_ym,
    country_code,
    platform_code,
    platform_version,
    
    -- Users with IoT devices registered
    COUNT(DISTINCT CASE WHEN SIZE(iot_info) > 0 THEN mac_addr END) AS users_with_iot_devices,
    
    -- Users by each iot_spec
    COUNT(DISTINCT CASE WHEN EXISTS(
      SELECT 1 FROM LATERAL VIEW EXPLODE(iot_info) t AS iot WHERE iot.iot_spec = 'thinq'
    ) THEN mac_addr END) AS thinq_users,
    
    COUNT(DISTINCT CASE WHEN EXISTS(
      SELECT 1 FROM LATERAL VIEW EXPLODE(iot_info) t AS iot WHERE iot.iot_spec = 'uei'
    ) THEN mac_addr END) AS uei_users,
    
    COUNT(DISTINCT CASE WHEN EXISTS(
      SELECT 1 FROM LATERAL VIEW EXPLODE(iot_info) t AS iot WHERE iot.iot_spec = 'ghp'
    ) THEN mac_addr END) AS ghp_users,
    
    COUNT(DISTINCT CASE WHEN EXISTS(
      SELECT 1 FROM LATERAL VIEW EXPLODE(iot_info) t AS iot WHERE iot.iot_spec = 'matter'
    ) THEN mac_addr END) AS matter_users
  
  FROM kic_data_mart.tv_usage.iot_usage
  WHERE date_ym IN ('2025-11', '2025-12', '2026-01')
    AND platform_version IN ('webOS22', 'webOS23', 'webOS24', 'webOS25')
  -- WHERE date_ym IN ('2025-12', '2026-01')
  --   AND platform_version IN ( 'webOS25')
  GROUP BY date_ym, country_code, platform_code, platform_version
)

-- Combine monthly usage with IoT metrics and add platform_id and region
SELECT 
  COALESCE(m.date_ym, i.date_ym) AS date_ym,
  COALESCE(m.country_code, i.country_code) AS country_code,
  cc.region,
  COALESCE(m.platform_version, i.platform_version) AS platform_version,
  COALESCE(m.platform_code, i.platform_code) AS platform_id,
  m.total_activated_tvs,
  i.users_with_iot_devices,
  i.thinq_users,
  i.uei_users,
  i.ghp_users,
  i.matter_users
FROM monthly_usage m
FULL OUTER JOIN iot_metrics i 
  ON m.date_ym = i.date_ym
  AND m.country_code = i.country_code 
  AND m.platform_code = i.platform_code
  AND m.platform_version = i.platform_version
LEFT JOIN kic_data_dimension.common.country_code cc
  ON COALESCE(m.country_code, i.country_code) = cc.country_code
ORDER BY date_ym, total_activated_tvs DESC

)

In [0]:
%sql
create or replace table sandbox.z_jeongjoo_lee.heds_1266_iot_usage_dev as (
WITH platform_id_mapping AS (
  -- Get platform_id from normal_log tables
  SELECT DISTINCT 
    X_Device_Platform AS platform_id,
    X_Device_Platform AS platform_code
  FROM (
    select distinct platform_code as X_Device_Platform
    from kic_data_fact.common_tv.tv_usage_monthly
  )
  WHERE X_Device_Platform IS NOT NULL
),
platform_mapping AS (
  -- Get platform_code to platform_version mapping
  SELECT DISTINCT 
    platform_code,
    platform_version
  FROM kic_data_mart.tv_usage.iot_usage
  WHERE date_ym IN ('2025-11', '2025-12', '2026-01')
    AND platform_version IN ('webOS22', 'webOS23', 'webOS24', 'webOS25')
),

iot_device_counts AS (
  -- Count devices by iot_spec and iot_type
  SELECT 
    date_ym,
    country_code,
    platform_code,
    platform_version,
    iot.iot_spec,
    iot.iot_type,
    COUNT(*) AS device_count
  FROM kic_data_mart.tv_usage.iot_usage
  LATERAL VIEW EXPLODE(iot_info) t AS iot
  WHERE date_ym IN ('2025-11', '2025-12', '2026-01')
    AND platform_version IN ('webOS22', 'webOS23', 'webOS24', 'webOS25')
  GROUP BY date_ym, country_code, platform_code, platform_version, iot.iot_spec, iot.iot_type
)

-- Device counts with region information
SELECT 
  d.date_ym,
  d.country_code,
  cc.region,
  d.platform_version,
  d.platform_code AS platform_id,
  d.iot_spec,
  d.iot_type,
  d.device_count
FROM iot_device_counts d
LEFT JOIN kic_data_dimension.common.country_code cc
  ON d.country_code = cc.country_code
ORDER BY date_ym, device_count DESC, iot_spec, iot_type

)

In [0]:
%sql
create table if not exists sandbox.z_jeongjoo_lee.heds_1266_tv_usage_monthly as (
  select *
  from kic_data_fact.common_tv.tv_usage_monthly
)

In [0]:
%sql
sandbox.z_jeongjoo_lee.heds_1266_iot_usage_dev
sandbox.z_jeongjoo_lee.heds_1266_iot_user_cnt_dev
sandbox.z_jeongjoo_lee.heds_1266_tv_usage_monthly
aic_nonpii_share.z_jeongjoo_lee.heds_1266_iot_usage_dev
aic_nonpii_share.z_jeongjoo_lee.heds_1266_iot_user_cnt_dev
aic_nonpii_share.z_jeongjoo_lee.heds_1266_tv_usage_monthly


In [0]:
%sql


In [0]:
%sql
create or replace view sandbox.z_jeongjoo_lee.v_global_heds_1266_iot_usage_dev as (
  select 
    'KIC' as workspace, a.* except (region)
  from sandbox.z_jeongjoo_lee.heds_1266_iot_usage_dev a

  union all 

  select 
    'AIC' as workspace, b.* except (region)
  from aic_nonpii_share.z_jeongjoo_lee.heds_1266_iot_usage_dev b

  union all

  select 
    'EIC' as workspace, c.* except (region)
  from eic_nonpii_share.z_jeongjoo_lee.heds_1266_iot_usage_dev c
)
;


In [0]:
%sql
create or replace view sandbox.z_jeongjoo_lee.v_global_heds_1266_iot_user_cnt_dev as (
  select 
    'KIC' as region, a.* except (region)
  from sandbox.z_jeongjoo_lee.heds_1266_iot_user_cnt_dev a

  union all 

  select 
    'AIC' as region, b.* except (region)
  from aic_nonpii_share.z_jeongjoo_lee.heds_1266_iot_user_cnt_dev b

  union all

  select 
    'EIC' as region, c.* except (region)
  from eic_nonpii_share.z_jeongjoo_lee.heds_1266_iot_user_cnt_dev c
)
;

In [0]:
%sql
create or replace view sandbox.z_jeongjoo_lee.v_global_heds_1266_tv_usage_monthly as (
  select 
    'KIC' as region, a.*
  from sandbox.z_jeongjoo_lee.heds_1266_tv_usage_monthly a

  union all 

  select 
    'AIC' as region, b.*
  from aic_nonpii_share.z_jeongjoo_lee.heds_1266_tv_usage_monthly b

  union all

  select 
    'EIC' as region, c.*
  from eic_nonpii_share.z_jeongjoo_lee.heds_1266_tv_usage_monthly c
)
;

In [0]:
%sql
select *
from 